In [2]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Activation, Dense, Input
from tensorflow.keras.layers import Reshape
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from keras import regularizers

In [3]:
def load_data(fname):
    data = pd.read_csv(fname, index_col=0)
    X = data.values[:, 2:].astype('float64')
    years = data['year']
    X_train = X[years < 2020.]
    X_valid = X[years == 2020.]
    tmp = data.index[data['year'] == 2020.]
    tickers = np.array([ticker.rstrip('_2020') for ticker in tmp])
    return X_train, X_valid, tickers

X_train, X_valid, tickers = load_data('assign2_data.csv')

In [4]:
def create_autoencoder(k_reg=0.01):
    input_shape = X_train.shape[1]
    latent_dim = 5
    inputs = Input(shape=input_shape, name='encoder_input')
    x = inputs
    latent = Dense(latent_dim, activation='relu', kernel_regularizer=regularizers.l2(k_reg))(x)
    encoder = Model(inputs, latent, name='encoder')
    latent_inputs = Input(shape=(latent_dim,), name='decoder_input')
    outputs = Dense(input_shape, activation='sigmoid', kernel_regularizer=regularizers.l2(k_reg))(latent_inputs)
    decoder = Model(latent_inputs, outputs, name='decoder')
    autoencoder = Model(inputs, decoder(encoder(inputs)), name='autoencoder')
    autoencoder.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    return autoencoder

In [5]:
autoencoder = create_autoencoder()

In [6]:
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 250)]             0         
_________________________________________________________________
encoder (Functional)         (None, 5)                 1255      
_________________________________________________________________
decoder (Functional)         (None, 250)               1500      
Total params: 2,755
Trainable params: 2,755
Non-trainable params: 0
_________________________________________________________________


In [7]:
# Train the autoencoder
autoencoder.fit(X_train,
                X_train,
                validation_data=(X_valid, X_valid),
                epochs=50,
                batch_size=32)

Epoch 1/50
15/15 [==============================] - 1s 14ms/step - loss: 0.2518 - accuracy: 0.0042 - val_loss: 0.2312 - val_accuracy: 0.0169
Epoch 2/50
15/15 [==============================] - 0s 3ms/step - loss: 0.2041 - accuracy: 0.0021 - val_loss: 0.1899 - val_accuracy: 0.0254
Epoch 3/50
15/15 [==============================] - 0s 4ms/step - loss: 0.1675 - accuracy: 0.0021 - val_loss: 0.1588 - val_accuracy: 0.0169
Epoch 4/50
15/15 [==============================] - 0s 4ms/step - loss: 0.1401 - accuracy: 0.0021 - val_loss: 0.1358 - val_accuracy: 0.0254
Epoch 5/50
15/15 [==============================] - 0s 5ms/step - loss: 0.1200 - accuracy: 0.0021 - val_loss: 0.1191 - val_accuracy: 0.0254
Epoch 6/50
15/15 [==============================] - 0s 5ms/step - loss: 0.1054 - accuracy: 0.0042 - val_loss: 0.1070 - val_accuracy: 0.0254
Epoch 7/50
15/15 [==============================] - 0s 4ms/step - loss: 0.0947 - accuracy: 0.0042 - val_loss: 0.0983 - val_accuracy: 0.0254
Epoch 8/50
15/15 [=

In [8]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=create_autoencoder, epochs=50, batch_size=32, verbose=0)
# define the grid search parameters
k_reg = [0.0001, 0.001, 0.01, 0.1, 1]
param_grid = dict(k_reg=k_reg)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_valid, X_valid)
# summarize results
best_lam = grid_result.best_params_['k_reg']
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


Best: 0.086957 using {'k_reg': 0.0001}


In [9]:
best_lam

0.0001

In [10]:
autoencoder = create_autoencoder(best_lam)

In [11]:
predict = autoencoder.predict(X_valid)

In [12]:
from numpy import array
from numpy.linalg import norm
communiality = array(np.zeros(len(X_valid)))
for i in range(len(X_valid)):
    communiality[i] = norm(array(X_valid)[i]-predict[i])

In [13]:
communiality

array([4.08934191, 6.85171714, 6.68162695, 6.932473  , 3.38658645,
       4.8810154 , 5.27149366, 6.26385206, 1.66428906, 1.39023996,
       4.62757477, 3.74397924, 5.23273418, 3.51859927, 4.54265336,
       2.96440797, 4.1971262 , 2.54323065, 4.02050054, 3.23208702,
       6.89016847, 5.45030774, 7.60892143, 5.37338104, 2.46407622,
       2.26225311, 7.49627105, 3.58722981, 7.00245641, 7.1921063 ,
       5.27958944, 6.48629982, 3.00441172, 4.44087334, 4.03678885,
       3.05178923, 1.67888844, 5.38501008, 3.21686141, 1.8051925 ,
       3.07419929, 1.93351073, 2.86521631, 2.77046442, 2.14090447,
       2.79501226, 3.57026206, 4.26833399, 2.36637164, 6.81906266,
       4.08345697, 5.93067145, 5.4431129 , 5.94240759, 4.08199115,
       7.62944005, 2.45038694, 7.71955193, 3.45085832, 3.29161738,
       6.84598953, 1.34345407, 3.29205816, 4.5167275 , 5.83351524,
       4.17229208, 5.69444699, 2.87789061, 5.18945264, 5.8249746 ,
       4.59639564, 7.67014338, 7.39881441, 1.26088877, 6.95764

In [17]:
top_5_idx = np.argsort(communiality)[0:5]
top_5_values = [communiality[i] for i in top_5_idx]
bottom_20_idx = np.argsort(communiality)[-20:]
bottom_20_values = [communiality[i] for i in bottom_20_idx]

print(top_5_idx)
print(top_5_values)
print(bottom_20_idx)
print(bottom_20_values)

[73 61  9 99  8]
[1.2608887729765725, 1.3434540731898188, 1.3902399582896636, 1.4440045351686008, 1.664289061352136]
[ 85  49 112  60   1  20   3  74  28  81  29  72 104  26 101  78  22  55
  71  57]
[6.795720265170573, 6.819062658611424, 6.820495079567932, 6.845989530071157, 6.851717143982328, 6.890168474690829, 6.932472995093266, 6.957644428523128, 7.002456409091976, 7.181463373594447, 7.192106299929343, 7.3988144135601, 7.432411361838834, 7.4962710529346, 7.521809308546173, 7.574685281988244, 7.608921432560085, 7.62944004976981, 7.670143383692932, 7.719551927838591]


In [18]:
Most_communal = tickers[top_5_idx]
Least_communal = tickers[bottom_20_idx]

In [19]:
print("Most communal:\n-----------------")
print(Most_communal)
print()
print("Least communal:\n--------------")
print(Least_communal)
print("\n")

Most communal:
-----------------
['IONS' 'PRAH' 'FGEN' 'CARA' 'ILMN']

Least communal:
--------------
['PRTA' 'GLYC' 'PRTK' 'SPPI' 'ALKS' 'DBVT' 'TXMD' 'RDUS' 'LXRX' 'ADMS'
 'CRIS' 'CLVS' 'MNKD' 'CMRX' 'ENDP' 'TLGT' 'PBYI' 'CLDX' 'BLCM' 'ACOR']


